# Step 1 import data

In [1]:
import pandas as pd 
import re 

#first file, in subfolder "data", use "\t" which does not exist in the data
#to ensure each row becomes a single cell. Name single column "raw" for raw text.
df1 = pd.read_csv(r"Data\TravisRolls_ Petals to the Metal.txt", sep="\t", names=["raw"]) 
df1.head()

,raw
0,Petals to the Metal
1,Episode 1 (3 rolls)
2,13 + 0 Wisdom save for Stephen
3,17 + 0 Investigation check
4,4 + 2 Dexterity save


## Concatenate multiple text inputs to one dataset

In [2]:
df1["season"] = 3 #label the seasons too

#Extract same data from other two text files, 
#representing two more seasons of recorded data.
df1_2 = pd.read_csv(r"Data\TravisRolls_ The Crystal Kingdom.txt", sep="\t", names=["raw"])
df1_2["season"] = 4
df1_3 = pd.read_csv(r"Data\TravisRolls_ The Eleventh Hour.txt", sep="\t", names=["raw"])
df1_3["season"] = 5

#Concatenate them all together into one dataframe
df1_concat = pd.concat([df1, df1_2, df1_3]).reset_index()
print(len(df1_concat.index))
df1_concat.head(100)

229


,index,raw,season
0,0,Petals to the Metal,3
1,1,Episode 1 (3 rolls),3
2,2,13 + 0 Wisdom save for Stephen,3
3,3,17 + 0 Investigation check,3
4,4,4 + 2 Dexterity save,3
...,...,...,...
95,12,Episode 3 (13 non-joke rolls),4
96,13,14 + 2 Dexterity Check,4
97,14,8 + 2 Initiative,4
98,15,10 + 1 Charisma Check,4


# Step 2: extract episode values and fill

In [3]:
#always make a copy to ensure I can work backwards if necessary
df2 = df1_concat.copy()

#Extract where the beginning of the line is "Episode #"
df2["episode"] = df2["raw"].str.extract("^Episode (\d)")
print(df2.head(5))

#The pad method fills subsequent NaNs with the last value that preceeded it
df2["episode"] = df2["episode"].fillna(method='pad')
df2.head(15)

   index                             raw  season episode
0      0            Petals to the Metal        3     NaN
1      1             Episode 1 (3 rolls)       3       1
2      2  13 + 0 Wisdom save for Stephen       3     NaN
3      3      17 + 0 Investigation check       3     NaN
4      4            4 + 2 Dexterity save       3     NaN


,index,raw,season,episode
0,0,Petals to the Metal,3,NaN
1,1,Episode 1 (3 rolls),3,1
2,2,13 + 0 Wisdom save for Stephen,3,1
3,3,17 + 0 Investigation check,3,1
4,4,4 + 2 Dexterity save,3,1
5,5,Episode 2 (8 rolls),3,2
6,6,16 + 7 Melee attack,3,2
7,7,"16 + 7 Melee attack (w/ advantage, never heard...",3,2
8,8,18 + 7 Melee attack,3,2
9,9,4 + 2 Dexterity Check to open lock,3,2


# Step 3 Extract content w Regex

In [4]:
df3 = df2.copy()

#Using regular expressions, designate groups of text to extract for further analysis
extraction_regex = r"^(\d+)\s+(?:\+\s+)?(\(\watural[ A-Za-z,]*?\)|\d{,2})?\s+" \
    + "([A-Za-z\'’]+(?: Tools| of Hand)?)?\s?(\wheck|\wave|\wttack)?(?: ?\(?w\/ )?(\w+dvantage)?"

#Insert regex groups into a list of columns
df3[["roll", "mod", "basis", "type", "adv"]] = df3["raw"].str.extract(extraction_regex)

#Separately extract if any row mentions "joke" since those should be removed
df3["joke"] = df3["raw"].str.extract("([jJ]oke)")

df3.head(100)

,index,raw,season,episode,roll,mod,basis,type,adv,joke
0,0,Petals to the Metal,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Episode 1 (3 rolls),3,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,13 + 0 Wisdom save for Stephen,3,1,13,0,Wisdom,save,NaN,NaN
3,3,17 + 0 Investigation check,3,1,17,0,Investigation,check,NaN,NaN
4,4,4 + 2 Dexterity save,3,1,4,2,Dexterity,save,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,12,Episode 3 (13 non-joke rolls),4,3,NaN,NaN,NaN,NaN,NaN,joke
96,13,14 + 2 Dexterity Check,4,3,14,2,Dexterity,Check,NaN,NaN
97,14,8 + 2 Initiative,4,3,8,2,Initiative,NaN,NaN,NaN
98,15,10 + 1 Charisma Check,4,3,10,1,Charisma,Check,NaN,NaN


# Step 4 Extract Natural 1s or Natural 20s which have special rules

In [5]:
df3_2 = df3.dropna(subset=['mod'])

nat_test = df3_2[df3_2['mod'].str.contains("atur")]
nat_test

,index,raw,season,episode,roll,mod,basis,type,adv,joke
62,62,"20 (Natural, as far as I could tell) Strength ...",3,9,20,"(Natural, as far as I could tell)",Strength,Check,NaN,NaN
91,8,20 (Natural) Intimidation Check,4,2,20,(Natural),Intimidation,Check,NaN,NaN
100,17,20 (natural) Melee Attack,4,3,20,(natural),Melee,Attack,NaN,NaN
142,59,1 (natural) Melee Attack,4,9,1,(natural),Melee,Attack,NaN,NaN
144,61,1 (natural) Dexterity Save,4,9,1,(natural),Dexterity,Save,NaN,NaN
145,62,"1 (natural) Dexterity Save (w/ disadvantage, n...",4,9,1,(natural),Dexterity,Save,disadvantage,NaN
149,66,"20 (natural) Wisdom Save (w/ advantage, never ...",4,9,20,(natural),Wisdom,Save,advantage,NaN


In [6]:
df4 = df3_2.copy()
df4['nat'] = df4['mod'].str.contains("atur")
df4['mod'] = df4.apply(lambda x: 0 if x['nat']==1 else x['mod'], axis=1)
df4.loc[list(nat_test.index)]

,index,raw,season,episode,roll,mod,basis,type,adv,joke,nat
62,62,"20 (Natural, as far as I could tell) Strength ...",3,9,20,0,Strength,Check,NaN,NaN,True
91,8,20 (Natural) Intimidation Check,4,2,20,0,Intimidation,Check,NaN,NaN,True
100,17,20 (natural) Melee Attack,4,3,20,0,Melee,Attack,NaN,NaN,True
142,59,1 (natural) Melee Attack,4,9,1,0,Melee,Attack,NaN,NaN,True
144,61,1 (natural) Dexterity Save,4,9,1,0,Dexterity,Save,NaN,NaN,True
145,62,"1 (natural) Dexterity Save (w/ disadvantage, n...",4,9,1,0,Dexterity,Save,disadvantage,NaN,True
149,66,"20 (natural) Wisdom Save (w/ advantage, never ...",4,9,20,0,Wisdom,Save,advantage,NaN,True


# Step 4: Condense to datapoints

In [7]:
df5 = df4[["season", "episode", "roll", "mod", "basis", "type", "adv","joke", "nat"]].copy()
df5 = df5[(df5['roll'].notnull()) & (pd.isna(df5['joke']))]
df5 = df5.drop(columns=["joke"])
def check_adv(x):
    x = str(x)
    if "dis" in x.lower():
        return -1
    elif "adv" in x.lower():
        return 1
    else:
        return 0
df5['adv'] = df5['adv'].apply(check_adv)
df5['basis'] = df5['basis'].str.lower()
df5['type'] = df5['type'].str.lower()
df5.head(20)

,season,episode,roll,mod,basis,type,adv,nat
2,3,1,13,0,wisdom,save,0,False
3,3,1,17,0,investigation,check,0,False
4,3,1,4,2,dexterity,save,0,False
6,3,2,16,7,melee,attack,0,False
7,3,2,16,7,melee,attack,1,False
8,3,2,18,7,melee,attack,0,False
9,3,2,4,2,dexterity,check,0,False
10,3,2,11,4,strength,check,0,False
11,3,2,16,2,constitution,check,0,False
12,3,2,13,7,athletics,check,0,False


# Step 5: Export usable dataset

In [8]:
df5.to_csv("TravisRollsDataset.csv")